In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/NNDL/Project/ARDL

/content/drive/MyDrive/Colab Notebooks/NNDL/Project/ARDL


In [3]:
%load_ext autoreload
%autoreload 2

from dataset import *

In [4]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)


In [8]:
emb_mat, data = create_inp_data_and_weights(newsgroups_train, newsgroups_test, glove_dim=50)

Vocabulary size [Bag-of-words]:  151353
Vocabulary size [GloVe]:  400000


In [25]:
from cnn import *
input_size, emb_size = emb_mat.shape

model = build_random_cnn(3, 5, 128, 256, input_size, emb_size, 7, emb_mat)
model

CNN(
  (embedding): Embedding(151353, 50)
  (conv_layers): Sequential(
    (0): Conv1d(50, 196, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): Dropout(p=0.5, inplace=False)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(196, 158, kernel_size=(5,), stride=(1,), padding=(2,))
    (5): Dropout(p=0.5, inplace=False)
    (6): ReLU()
    (7): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (8): Conv1d(158, 134, kernel_size=(5,), stride=(1,), padding=(2,))
    (9): Dropout(p=0.5, inplace=False)
    (10): ReLU()
    (11): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (12): Conv1d(134, 252, kernel_size=(5,), stride=(1,), padding=(2,))
    (13): Dropout(p=0.5, inplace=False)
    (14): ReLU()
    (15): AdaptiveMaxPool1d(output_size=20)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=5040, out_features=256, bias=True)
  (fc2): Lin

In [26]:
learning_rate = 0.001
num_epochs = 10
batch_size = 128
display_step = 4

criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [27]:
from torch.autograd import Variable

In [28]:
loss_l = []
for epoch in range(num_epochs):
    total_batch = int(len(newsgroups_train.data)/batch_size)
    loss_ep = []
    # Loop over all batches
    for i in range(total_batch):
        batch_x,batch_y = get_batch(data['train'],i,batch_size)

        articles = Variable(torch.LongTensor(batch_x))
        labels = Variable(torch.LongTensor(batch_y))
        # print("articles",articles)
        # print(batch_x, labels)
        # print("size labels",labels.size())

        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = model(articles)
        #print(outputs.shape, labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        loss_ep.append(loss.data)

        if (i+1) % display_step == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(newsgroups_train.data)//batch_size, loss.data))    
    loss_l.append(np.mean(loss_ep))

torch.Size([128, 500, 1, 50])
torch.Size([128, 500, 50])


/content/drive/MyDrive/Colab Notebooks/NNDL/Project/ARDL/cnn.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


torch.Size([128, 500, 1, 50])
torch.Size([128, 500, 50])
torch.Size([128, 500, 1, 50])
torch.Size([128, 500, 50])
torch.Size([128, 500, 1, 50])
torch.Size([128, 500, 50])
Epoch [1/10], Step [4/88], Loss: 1.8534
torch.Size([128, 500, 1, 50])
torch.Size([128, 500, 50])
torch.Size([128, 500, 1, 50])
torch.Size([128, 500, 50])
torch.Size([128, 500, 1, 50])
torch.Size([128, 500, 50])
torch.Size([128, 500, 1, 50])
torch.Size([128, 500, 50])
Epoch [1/10], Step [8/88], Loss: 1.8217
torch.Size([128, 500, 1, 50])
torch.Size([128, 500, 50])
torch.Size([128, 500, 1, 50])
torch.Size([128, 500, 50])
torch.Size([128, 500, 1, 50])
torch.Size([128, 500, 50])
torch.Size([128, 500, 1, 50])
torch.Size([128, 500, 50])
Epoch [1/10], Step [12/88], Loss: 1.6800
torch.Size([128, 500, 1, 50])
torch.Size([128, 500, 50])
torch.Size([128, 500, 1, 50])
torch.Size([128, 500, 50])
torch.Size([128, 500, 1, 50])
torch.Size([128, 500, 50])
torch.Size([128, 500, 1, 50])
torch.Size([128, 500, 50])
Epoch [1/10], Step [16/8

KeyboardInterrupt: ignored

In [31]:
# # Training accuracy
# correct = 0
# total = 0
# total_train_data = len(data['train']['target'])
# batch_x_train,batch_y_train = get_batch(data['train'],0,total_train_data)
# articles = Variable(torch.LongTensor(batch_x_train))
# labels = torch.LongTensor(batch_y_train)
# outputs = model(articles)
# _, predicted = torch.max(outputs.data, 1)
# total += labels.size(0)
# correct += (predicted == labels).sum()
# print('Accuracy of the network on the train articles: %d %%' % (100 * correct / total))
# train_accuracy_l.append(100 * correct / total)
test_accuracy_l = []
# # Test accuracy
correct = 0
total = 0
total_test_data = len(data['test']['target'])
batch_x_test,batch_y_test = get_batch(data['test'],0,batch_size)
articles = Variable(torch.LongTensor(batch_x_test))
labels = torch.LongTensor(batch_y_test)
outputs = model(articles)
_, predicted = torch.max(outputs.data, 1)
total += labels.size(0)
correct += (predicted == labels).sum()
print('Accuracy of the network on the test articles: %d %%' % (100 * correct / total))
test_accuracy_l.append(100 * correct / total)      

Accuracy of the network on the test articles: 79 %


/content/drive/MyDrive/Colab Notebooks/NNDL/Project/ARDL/cnn.py:55: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
from matplotlib import pyplot as plt
plt.plot(list(range(num_epochs)), loss_l)
plt.plot(list(range(num_epochs)), accuracy_l)
plt.show()